<h3>Sin Cython</h3>
<p>Este programa genera $N$ enteros aleatorios entre $1$ y $M$, y una vez obtenidos los&nbsp; eleva al cuadrado y devuelve la suma de los cuadrados. Por tanto, calcula el cuadrado de la longitud&nbsp; de un vector aleatorio con coordenadas enteros en el intervalo $[1,M]$.</p>

In [1]:
def cuadrados(N,M):
    res = 0
    for muda in xrange(N):
        x = randint(1,M)
        res += x*x
    return res

In [2]:
for n in srange(3,8):
    time A = cuadrados(10^n,10^6)

Time: CPU 0.01 s, Wall: 0.01 s
Time: CPU 0.05 s, Wall: 0.05 s
Time: CPU 0.48 s, Wall: 0.48 s
Time: CPU 4.83 s, Wall: 4.83 s
Time: CPU 48.22 s, Wall: 48.23 s

<h3>Con Cython</h3>
<p>Mismo c&aacute;lculo:</p>

In [3]:
%cython
import math
import random
def cuadrados_cy(long long N, long long M):
    cdef long long res = 0
    cdef long long muda
    cdef long long x
    for muda in xrange(N):
        x = random.randint(1,M)
        res += math.pow(x,2)
    return res

In [4]:
for n in srange(3,8):
    time A = cuadrados_cy(10^n,10^6)

Time: CPU 0.00 s, Wall: 0.00 s
Time: CPU 0.02 s, Wall: 0.02 s
Time: CPU 0.23 s, Wall: 0.23 s
Time: CPU 2.28 s, Wall: 2.29 s
Time: CPU 22.99 s, Wall: 22.99 s

<h3>Optimizando el c&aacute;lculo de n&uacute;meros aleatorios:</h3>

In [5]:
%cython
##############################################################
##Este primer trozo genera enteros aleatorios entre 1$ y $10^6$ usando 
#librerías externas compilables en C--Este trozo se puede reutilizar
##################################################################
cdef extern from "gsl/gsl_rng.h":
   ctypedef struct gsl_rng_type:
      pass
   ctypedef struct gsl_rng:
       pass
   gsl_rng_type *gsl_rng_mt19937
   gsl_rng *gsl_rng_alloc(gsl_rng_type * T)
  
cdef gsl_rng *r = gsl_rng_alloc(gsl_rng_mt19937)

cdef extern from "gsl/gsl_randist.h":
     long int uniform "gsl_rng_uniform_int"(gsl_rng * r, unsigned long int n)

def main():
    cdef int n
    n = uniform(r,1000000)
    return n
#########################################################################
#########################################################################
cdef long f(long x):
    return x**2
       
import random
def cuadrados_cy2(int N):
    cdef long res = 0
    cdef int muda
    for muda in range(N):
        res += f(main())
    return res

In [6]:
for n in srange(3,8):
    time A = cuadrados_cy2(10^n)

Time: CPU 0.00 s, Wall: 0.00 s
Time: CPU 0.00 s, Wall: 0.00 s
Time: CPU 0.00 s, Wall: 0.01 s
Time: CPU 0.05 s, Wall: 0.05 s
Time: CPU 0.52 s, Wall: 0.52 s

<h3>Problema similar sin n&uacute;meros aleatorios:</h3>

In [7]:
%cython
def cuadrados_cy3(long long int N):
    cdef long long int res = 0
    cdef long long int k
    for k in range(N):
        res += k**2
    return res

In [8]:
for n in srange(3,8):
    time A = cuadrados_cy3(10^n)

Time: CPU 0.00 s, Wall: 0.00 s
Time: CPU 0.00 s, Wall: 0.00 s
Time: CPU 0.00 s, Wall: 0.00 s
Time: CPU 0.00 s, Wall: 0.00 s
Time: CPU 0.01 s, Wall: 0.01 s

In [9]:
def cuadrados5(N):
    res = 0
    for k in range(N):
        res += k**2
    return res

In [10]:
for n in srange(3,8):
    time A = cuadrados5(10^n)

Time: CPU 0.00 s, Wall: 0.00 s
Time: CPU 0.00 s, Wall: 0.00 s
Time: CPU 0.03 s, Wall: 0.03 s
Time: CPU 0.32 s, Wall: 0.32 s
Time: CPU 3.27 s, Wall: 3.27 s

<p>Hemos comprobado, de dos maneras, que es en la generaci&oacute;n&nbsp; de los n&uacute;meros&nbsp; aleatorios donde Python pasa la mayor parte del tiempo en este c&aacute;lculo. Si optimizamos esa parte, usando una librer&iacute;a en C, o simplemente la suprimimos, el c&aacute;lculo es mucho m&aacute;s r&aacute;pido.&nbsp; Cython pierde much&iacute;sima eficiencia cuando debe ejecutar funciones de Python que son mucho m&aacute;s lentas que las correspondientes funciones en C.</p>